In [1]:
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.losses import MeanAbsolutePercentageError as MAPE
import tensorflow as tf
import pickle
import keras
from keras import layers, models, regularizers, backend, utils
import keras.backend.tensorflow_backend as tfback
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge
from sklearn.multioutput import MultiOutputRegressor


Using TensorFlow backend.


In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
TEST_RATIO = 0.2

In [4]:
mape = MAPE()

files = ["/notebooks/WECs_DataSet/Sydney_Data.csv"]
csvfile = open(files[0])
reader = csv.reader(csvfile, delimiter = ',')
rows = [[float(v) for v in row] for row in reader]
rows = np.asarray(rows, dtype='float32')

In [5]:
xs = rows[:,0:32]
xs = xs / np.max(xs)
ys = rows[:,32:49]
ys = ys / np.max(ys)
xs_train, xs_test, ys_train, ys_test = train_test_split(xs,ys,test_size = TEST_RATIO)
print(xs_train.shape,ys_train.shape)
print(xs_test.shape,ys_test.shape)

(57600, 32) (57600, 17)
(14400, 32) (14400, 17)


In [ ]:
regression_model = LinearRegression()
regression_model.fit(xs_train,ys_train)
ys_predict = regression_model.predict(xs_test)
mape(ys_test,ys_predict).numpy()

In [ ]:
K = keras.backend

hyperparamter_penalty_weight = 0.1

def custom_loss(y_true,y_pred):
  return K.mean(K.square(y_pred - y_true), axis=-1) + hyperparamter_penalty_weight * K.mean(K.square(K.sum(y_pred[:,0:16], axis=1) - y_pred[:,16]), axis=-1)

def penalty(y_true,y_pred):
  return 100*K.mean(K.abs(K.sum(y_pred[:,0:16], axis=1) - y_pred[:,16])/K.abs(y_pred[:,16]), axis=-1)

inputs = layers.Input(shape=(32,))
d1 = layers.Dense(32, activation = 'relu')(inputs)
d2 = layers.Dense(64, activation = 'relu')(d1)
d3 = layers.Dense(32, activation = 'relu')(d2)
outputs = layers.Dense(17, activation = 'linear')(d3)

simple_model = models.Model(inputs = inputs, outputs = outputs)
simple_model.compile('adam', custom_loss, metrics=['mean_absolute_percentage_error'])
simple_model.summary()

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_test_end(self, logs = None):
        clear_output()

loss_hist = simple_model.fit(xs_train, ys_train, epochs = 3000, validation_data = (xs_test, ys_test), batch_size=2000)

In [ ]:
ys_predictions = simple_model.predict(xs_test)
mape = tf.keras.losses.MeanAbsolutePercentageError()
mape(ys_predictions, ys_test).numpy()